In [63]:
import pandas as pd
import numpy as np

## Before getting started, Please copy Datasets/human_annotation directory to currency dir

## Fleiss kappa Calculation Reference  
Link: https://en.wikipedia.org/wiki/Fleiss%27_kappa

## 1.0 Chinese-English annotation preprocess

In [64]:
df_cn_en_1 = pd.read_csv("./human_annotation/Chinese-English/Chinese-English-human-annotation-1 - Sheet1.csv")
df_cn_en_2 = pd.read_csv("./human_annotation/Chinese-English/Chinese-English-human-annotation-2 - Sheet1.csv")
df_cn_en_3 = pd.read_csv("./human_annotation/Chinese-English/Chinese-English-human-annotation-3 - Sheet1.csv")
df_cn_en_4 = pd.read_csv("./human_annotation/Chinese-English/Chinese-English-human-annotation-4 - Sheet1.csv")
df_cn_en_5 = pd.read_csv("./human_annotation/Chinese-English/Chinese-English-human-annotation-5 - Sheet1.csv")
df_cn_en_6 = pd.read_csv("./human_annotation/Chinese-English/Chinese-English-human-annotation-6 - Sheet1.csv")
assert(len(df_cn_en_1)==len(df_cn_en_2))
assert(len(df_cn_en_3)==len(df_cn_en_4))
assert(len(df_cn_en_5)==len(df_cn_en_6))
assert(len(df_cn_en_1)==len(df_cn_en_3))
assert(len(df_cn_en_3)==len(df_cn_en_5))
SubjectNum = len(df_cn_en_1)

In [65]:
cn_en_extracted_col_from2 = df_cn_en_2.rename(columns={"Is loan?": "Is loan?2"})["Is loan?2"]
cn_en_extracted_col_from3 = df_cn_en_3.rename(columns={"Is loan?": "Is loan?3"})["Is loan?3"]
cn_en_extracted_col_from4 = df_cn_en_4.rename(columns={"Is loan?": "Is loan?4"})["Is loan?4"]
cn_en_extracted_col_from5 = df_cn_en_5.rename(columns={"Is loan?": "Is loan?5"})["Is loan?5"]
cn_en_extracted_col_from6 = df_cn_en_6.rename(columns={"Is loan?": "Is loan?6"})["Is loan?6"]
df_all_raters_cn_en = df_cn_en_1.join(cn_en_extracted_col_from2).join(cn_en_extracted_col_from3).join(cn_en_extracted_col_from4)
df_all_raters_cn_en = df_all_raters_cn_en.join(cn_en_extracted_col_from5).join(cn_en_extracted_col_from6).fillna(0)
df_all_raters_cn_en = df_all_raters_cn_en.replace(to_replace=r'[xX]', value=1, regex=True)
df_all_raters_cn_en

,Unnamed: 0,Word,Is loan?,Is loan?2,Is loan?3,Is loan?4,Is loan?5,Is loan?6
0,101,印度尼西亞,1,1,1,1,1,1
1,895,阿爾弗列德,1,1,0,1,1,1
2,183,喹啉,1,1,0,1,1,1
3,49,免治,1,1,0,1,0,1
4,161,咪唑,1,0,0,1,1,1
...,...,...,...,...,...,...,...,...
90,442,拓都,1,1,0,1,1,1
91,112,可待因,1,1,0,1,1,1
92,476,方棚,1,0,0,1,0,1
93,855,那不勒斯,1,1,1,1,1,1


## 1.1 Chinese-English annotation accuracy

In [38]:
sum_loan_df = df_all_raters_cn_en.drop(columns=['Unnamed: 0', 'Word']).sum(axis=0,numeric_only= True).to_frame()
sum_loan_df.sum().to_frame()/(95*6)

,0
0,0.852632


## check how many words in spreadsheet are also in the Chinese-English false friends list

In [68]:
df_false_friend = pd.read_csv("./human_annotation/Chinese-English/Chinese-English_false_friends.csv")
df_false_friend_lst = list(df_false_friend["loan_word"])
df_all_raters_cn_en_lst = list(df_all_raters_cn_en["Word"])
count_cn_en_false_friend = [word for word in df_all_raters_cn_en_lst if word in df_false_friend_lst]
count_cn_en_false_friend
# count_cn_en_false_friend

['塞特']

# *Fleiss kappa main function (Reload this function for other language pairs)*

In [66]:
def CalculatePi(row, RaterNum):
    numerator = row['loan_num']**2 + row['not_loan_num']**2 - RaterNum
    denominator = RaterNum*(RaterNum-1)
    return numerator/denominator

# fleiss kappa function
def calculateFleissKappa(RaterNum, df_all_raters):
    SubjectNum = len(df_all_raters)
    print("RaterNum: ", RaterNum)
    print("SubjectNum", SubjectNum)
    
    df_without_loan_count = df_all_raters.copy()
    df_without_loan_count["loan_num"] = df_without_loan_count.drop(columns=['Unnamed: 0', 'Word']).sum(axis=1,numeric_only= True)
    df_without_loan_count['not_loan_num'] = df_without_loan_count.apply(lambda row: RaterNum -row['loan_num'], axis=1)
    print("Check intermediate df",df_without_loan_count)
    df_without_loan_count['Pi'] = df_without_loan_count.apply(lambda row: CalculatePi(row, RaterNum), axis=1)
    
    Pj_loan = df_without_loan_count['loan_num'].sum()/(SubjectNum*RaterNum)
    Pj_not_loan = df_without_loan_count['not_loan_num'].sum()/(SubjectNum*RaterNum)
    Pe_avg = Pj_loan**2 + Pj_not_loan**2
    Pi_avg = df_without_loan_count['Pi'].sum()/SubjectNum
    kappa = (Pi_avg-Pe_avg)/(1-Pe_avg)
    return kappa

##  1.2 Chinese-English annotation fleiss kappa

In [67]:
calculateFleissKappa(6, df_all_raters_cn_en)

RaterNum:  6
SubjectNum 95
Check intermediate df     Unnamed: 0   Word  Is loan?  Is loan?2  Is loan?3  Is loan?4  Is loan?5  \
0          101  印度尼西亞         1          1          1          1          1   
1          895  阿爾弗列德         1          1          0          1          1   
2          183     喹啉         1          1          0          1          1   
3           49     免治         1          1          0          1          0   
4          161     咪唑         1          0          0          1          1   
..         ...    ...       ...        ...        ...        ...        ...   
90         442     拓都         1          1          0          1          1   
91         112    可待因         1          1          0          1          1   
92         476     方棚         1          0          0          1          0   
93         855   那不勒斯         1          1          1          1          1   
94         483     智利         1          1          1          1          1   

  

0.03380364491475685

## 2.0 English Persian annotation file preprocess

In [74]:
df_en_fr_1 = pd.read_csv("./human_annotation/English-French/English-French-human-annotation-1 - Sheet1.csv")
df_en_fr_2 = pd.read_csv("./human_annotation/English-French/English-French-human-annotation-2 - Sheet1.csv")
df_en_fr_3 = pd.read_csv("./human_annotation/English-French/English-French-human-annotation-3 - Sheet1.csv")
df_en_fr_4 = pd.read_csv("./human_annotation/English-French/English-French-human-annotation-4 - Sheet1.csv")
df_en_fr_5 = pd.read_csv("./human_annotation/English-French/English-French-human-annotation-5 - Sheet1.csv")
df_en_fr_6 = pd.read_csv("./human_annotation/English-French/English-French-human-annotation-6 - Sheet1.csv")
df_en_fr_7 = pd.read_csv("./human_annotation/English-French/English-French-human-annotation-7 - Sheet1.csv")
assert(len(df_en_fr_1)==len(df_en_fr_2))
assert(len(df_en_fr_3)==len(df_en_fr_4))
assert(len(df_en_fr_5)==len(df_en_fr_6))
assert(len(df_en_fr_7)==len(df_en_fr_1))
assert(len(df_en_fr_3)==len(df_en_fr_5))
SubjectNum = len(df_en_fr_1)

In [75]:
en_fr_extracted_col_from2 = df_en_fr_2.rename(columns={"Is loan?": "Is loan?2"})["Is loan?2"]
en_fr_extracted_col_from3 = df_en_fr_3.rename(columns={"Is loan?": "Is loan?3"})["Is loan?3"]
en_fr_extracted_col_from4 = df_en_fr_4.rename(columns={"Is loan?": "Is loan?4"})["Is loan?4"]
en_fr_extracted_col_from5 = df_en_fr_5.rename(columns={"Is loan?": "Is loan?5"})["Is loan?5"]
en_fr_extracted_col_from6 = df_en_fr_6.rename(columns={"Is loan?": "Is loan?6"})["Is loan?6"]
en_fr_extracted_col_from7 = df_en_fr_7.rename(columns={"Is loan?": "Is loan?7"})["Is loan?7"]
df_all_raters_en_fr = df_en_fr_1.join(en_fr_extracted_col_from2).join(en_fr_extracted_col_from3).join(en_fr_extracted_col_from4)
df_all_raters_en_fr = df_all_raters_en_fr.join(en_fr_extracted_col_from5).join(en_fr_extracted_col_from6).join(en_fr_extracted_col_from7).fillna(0)
df_all_raters_en_fr[["Is loan?","Is loan?2","Is loan?3","Is loan?4","Is loan?5","Is loan?6","Is loan?7"]] = df_all_raters_en_fr[["Is loan?","Is loan?2","Is loan?3","Is loan?4","Is loan?5","Is loan?6","Is loan?7"]].replace(to_replace=r'[xX]', value=1, regex=True)
df_all_raters_en_fr


,Unnamed: 0,Word,Is loan?,Is loan?2,Is loan?3,Is loan?4,Is loan?5,Is loan?6,Is loan?7
0,0,babul,1,1,1,1,1,1,0
1,7,detergent,0,0,0,0,0,0,0
2,12,tricycle,1,0,0,0,0,0,1
3,27,chef,1,0,1,1,1,0,1
4,34,capriole,1,1,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...
503,3468,bombard,0,0,1,0,1,0,1
504,3472,petit mal,1,1,1,1,1,0,0
505,3480,abbé,1,1,0,0,1,0,0
506,3481,mixity,0,0,0,1,0,0,0


## 2.1 English-French annotation accuracy

In [78]:
# accuracy
sum_loan_df = df_all_raters_en_fr.drop(columns=['Unnamed: 0', 'Word']).sum(axis=0,numeric_only= True).to_frame()

sum_loan_df.sum().to_frame()/(508*7)

,0
0,0.288526


## 2.2 English-French annotation fleiss kappa

In [79]:
# fleiss kappa
calculateFleissKappa(7, df_all_raters_en_fr)

RaterNum:  7
SubjectNum 508
Check intermediate df      Unnamed: 0       Word  Is loan?  Is loan?2  Is loan?3  Is loan?4  \
0             0      babul         1          1          1          1   
1             7  detergent         0          0          0          0   
2            12   tricycle         1          0          0          0   
3            27       chef         1          0          1          1   
4            34   capriole         1          1          0          1   
..          ...        ...       ...        ...        ...        ...   
503        3468    bombard         0          0          1          0   
504        3472  petit mal         1          1          1          1   
505        3480       abbé         1          1          0          0   
506        3481     mixity         0          0          0          1   
507        3489    lucerne         0          1          1          1   

    Is loan?5  Is loan?6 Is loan?7  loan_num  not_loan_num  
0           

0.05886785474886158

## 3.0 Hindi-Persian annotation preprocess

In [80]:
df_hi_pers_1 = pd.read_csv("./human_annotation/Hindi-Persian/Hindi-Persian-human-annotation-1 - Sheet1.csv")
df_hi_pers_2 = pd.read_csv("./human_annotation/Hindi-Persian/Hindi-Persian-human-annotation-2 - Sheet1.csv")
df_hi_pers_3 = pd.read_csv("./human_annotation/Hindi-Persian/Hindi-Persian-human-annotation-3 - Sheet1.csv")
df_hi_pers_4 = pd.read_csv("./human_annotation/Hindi-Persian/Hindi-Persian-human-annotation-4 - Sheet1.csv")
df_hi_pers_5 = pd.read_csv("./human_annotation/Hindi-Persian/Hindi-Persian-human-annotation-5 - Sheet1.csv")
df_hi_pers_6 = pd.read_csv("./human_annotation/Hindi-Persian/Hindi-Persian-human-annotation-6 - Sheet1.csv")
assert(len(df_hi_pers_1)==len(df_hi_pers_2))
assert(len(df_hi_pers_3)==len(df_hi_pers_4))
assert(len(df_hi_pers_5)==len(df_hi_pers_6))
assert(len(df_hi_pers_4)==len(df_hi_pers_6))
SubjectNum = len(df_hi_pers_1)

In [81]:
hi_pers_extracted_col_from2 = df_hi_pers_2.rename(columns={"Is loan?": "Is loan?2"})["Is loan?2"]
hi_pers_extracted_col_from3 = df_hi_pers_3.rename(columns={"Is loan?": "Is loan?3"})["Is loan?3"]
hi_pers_extracted_col_from4 = df_hi_pers_4.rename(columns={"Is loan?": "Is loan?4"})["Is loan?4"]
hi_pers_extracted_col_from5 = df_hi_pers_5.rename(columns={"Is loan?": "Is loan?5"})["Is loan?5"]
hi_pers_extracted_col_from6 = df_hi_pers_6.rename(columns={"Is loan?": "Is loan?6"})["Is loan?6"]
df_all_raters_hi_pers = df_hi_pers_1.join(hi_pers_extracted_col_from2).join(hi_pers_extracted_col_from3).join(hi_pers_extracted_col_from4)
df_all_raters_hi_pers = df_all_raters_hi_pers.join(hi_pers_extracted_col_from5).join(hi_pers_extracted_col_from6).fillna(0)
df_all_raters_hi_pers[["Is loan?","Is loan?2","Is loan?3","Is loan?4","Is loan?5","Is loan?6"]] = df_all_raters_hi_pers[["Is loan?","Is loan?2","Is loan?3","Is loan?4","Is loan?5","Is loan?6"]].replace(to_replace=r'[xX]', value=1, regex=True)
df_all_raters_hi_pers


,Unnamed: 0,Word,Is loan?,Is loan?2,Is loan?3,Is loan?4,Is loan?5,Is loan?6
0,3,अंदर,1,0,0,1,0,0
1,9,अगर,1,0,0,0,0,0
2,28,अब्र,1,1,1,1,1,1
3,58,आफ़त,1,1,1,1,0,1
4,59,आफ़ताब,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...
120,1214,हयात,1,0,1,1,0,1
121,1220,हलवा,1,0,0,1,0,0
122,1236,हिलाल,1,0,1,1,1,1
123,1243,हैरत,1,1,1,0,0,1


## 3.1 Hindi-Persian annotation accuracy

In [82]:
# accuracy
sum_loan_df = df_all_raters_hi_pers.drop(columns=['Unnamed: 0', 'Word']).sum(axis=0,numeric_only= True).to_frame()
sum_loan_df.sum().to_frame()/(125*6)

,0
0,0.598667


## Hindi-Persian annotation fleiss kappa

In [83]:
calculateFleissKappa(6, df_all_raters_hi_pers)

RaterNum:  6
SubjectNum 125
Check intermediate df      Unnamed: 0    Word  Is loan?  Is loan?2  Is loan?3  Is loan?4  Is loan?5  \
0             3    अंदर         1          0          0          1          0   
1             9     अगर         1          0          0          0          0   
2            28    अब्र         1          1          1          1          1   
3            58    आफ़त         1          1          1          1          0   
4            59  आफ़ताब         1          1          1          1          1   
..          ...     ...       ...        ...        ...        ...        ...   
120        1214    हयात         1          0          1          1          0   
121        1220    हलवा         1          0          0          1          0   
122        1236   हिलाल         1          0          1          1          1   
123        1243    हैरत         1          1          1          0          0   
124        1244   हैरान         1          0          0    

0.1132009855788799

## Extra experiment

In [54]:
def CalculatePi(row):
    numerator = row['loan_num']**2 + row['not_loan_num']**2 - RaterNum
    denominator = RaterNum*(RaterNum-1)
    return numerator/denominator
df_all_raters_cn_en['Pi'] = df_all_raters_cn_en.apply(lambda row: CalculatePi(row), axis=1)
df_all_raters_cn_en

,Unnamed: 0,Word,Is loan?,Is loan?2,Is loan?3,Is loan?4,Is loan?5,Is loan?6,loan_num,not_loan_num,Pi
0,101,印度尼西亞,1,1,1,1,1,1,6,0,1.000000
1,895,阿爾弗列德,1,1,0,1,1,1,5,1,0.666667
2,183,喹啉,1,1,0,1,1,1,5,1,0.666667
3,49,免治,1,1,0,1,0,1,4,2,0.466667
4,161,咪唑,1,0,0,1,1,1,4,2,0.466667
...,...,...,...,...,...,...,...,...,...,...,...
90,442,拓都,1,1,0,1,1,1,5,1,0.666667
91,112,可待因,1,1,0,1,1,1,5,1,0.666667
92,476,方棚,1,0,0,1,0,1,3,3,0.400000
93,855,那不勒斯,1,1,1,1,1,1,6,0,1.000000
